# <u>Bengals Data ETL</u>

### Establish Package Imports

In [13]:
import boto3
import pandas as pd
import io
from sqlalchemy import create_engine
import os

### Create Connection Class for AWS S3 Bucket

Notes:
- AWS Credentials were setup with Environment Variables
- 

Implementation:
There were many approaches I felt I could take with this. I decided implement this with the intent that maybe we might want to add additional teams to this if we threw in some more csvs. I broke it up into different classes as well to help breakdown some of my thought process & hopefully made it easier to follow.

In [5]:
class Data_Connection(object):
    def __init__(self):
        self.aws_connection = boto3.Session()
        self.s3_connection = self.aws_connection.client('s3')
        self.aws_bucket = 'mindex-data-analytics-code-challenge'
    
    def check_bucket_connection(self):
        print("Configurating Connection to S3 Bucket...")
        try:
            response = self.s3_connection.list_objects(Bucket=self.aws_bucket)
            print("Connection Established!")
        except Exception as e:
            print(e)
    
    def bucket_extract(self):
        
        #Get list of Data Sources in S3 Bucket
        response = self.s3_connection.list_objects(Bucket=self.aws_bucket)
        sources = response.get("Contents")
        
        
        #Loop through Data Sources found in S3 Bucket
        data_sources = {'player_data': {}, 'team_data': {}}
        
        print('Extracting Sources...')
        for source in sources:
            s3_obj = self.s3_connection.get_object(Bucket=self.aws_bucket, Key=source['Key'])
            nfl_df = pd.read_csv(io.BytesIO(s3_obj['Body'].read()))
            
            #Hold Dataframe in Dictionary to easier analysis
            source_name = source['Key'].split('.')[0]

            if "receiving" in source_name: 
                data_sources['player_data'][source_name] = nfl_df
            else:
                data_sources['team_data'][source_name] = nfl_df
            
            
        print('Extraction Completed!')
        return data_sources 

In [6]:
connection = Data_Connection()
connection.check_bucket_connection()

Configurating Connection to S3 Bucket...
Connection Established!


In [7]:
nfl_dataframes = connection.bucket_extract()

Extracting Sources...
Extraction Completed!


In [8]:
class Data_Transformation(object):
    def __init__(self,data_dict):
        self.trans_data = data_dict
        self.team_data = data_dict.get('team_data')
        self.player_data = data_dict.get('player_data')
        self.weekly_stats = None
    
    def transform_data(self):
        
        #transform team data to specifications
        team = self.team_data['bengals']
        team['Result'].replace(to_replace=1, value='Win', inplace=True)
        team['Result'].replace(to_replace=0, value='Loss', inplace=True)
        self.team_data = team
        print(self.team_data)
        
        #transform player data to specifications
        for player in self.player_data:    
            player_df = self.player_data[player]
            player_name = player.split('_')[0]
            player_df.rename(columns={"Yards": player_name + "_Yards", "TD": player_name + "_TD"}, inplace=True)
            self.player_data[player] = player_df
            print(player_df)
                
    
    def merge_data(self):
        self.weekly_stats = self.team_data
        for key in self.player_data:
            self.weekly_stats = pd.merge(self.weekly_stats, self.player_data[key], how='left')
        
        #remove preseason games due to no seasonal data
        self.weekly_stats = self.weekly_stats[~self.weekly_stats['Week'].str.contains('PRE')] 
        
        print(self.weekly_stats)
        return self.weekly_stats
        

In [9]:
trans = Data_Transformation(nfl_dataframes)
trans.transform_data()
transformed_data = trans.merge_data()

     Week Opponent Location Result
0    PRE1       TB     Away    Win
1    PRE2      WSH     Away   Loss
2    PRE3      MIA     Home   Loss
3    REG1      MIN     Home    Win
4    REG2      CHI     Away   Loss
5    REG3      PIT     Away    Win
6    REG4      JAX     Home    Win
7    REG5       GB     Home   Loss
8    REG6      DET     Away    Win
9    REG7      BAL     Away    Win
10   REG8      NYJ     Away   Loss
11   REG9      CLE     Home   Loss
12  REG10      NaN      NaN    NaN
13  REG11       LV     Away    Win
14  REG12      PIT     Home    Win
15  REG13      LAC     Home   Loss
16  REG14       SF     Home   Loss
17  REG15      DEN     Away    Win
18  REG16      BAL     Home    Win
19  REG17       KC     Home    Win
20  REG18      CLE     Away   Loss
21  POST1       LV     Home    Win
22  POST2      TEN     Away    Win
23  POST3       KC     Away    Win
24  POST4      LAR  Neutral   Loss
     Week  boyd_Yards  boyd_TD
0    REG1          32        0
1    REG2          73       

In [10]:
class Data_Loader(object):
    def __init__(self, data):
        self.loading_data = data
        
        self.dbusername = os.environ['MINDEXDB_USERNAME']
        self.dbpassword = os.environ['MINDEXDB_PASSWORD']
        self.dbhost = os.environ['MINDEXDB_HOST']
        self.dbport = 5432
        self.dbname = os.environ['MINDEXDB_DBNAME']
        self.dbtable = os.environ['MINDEXDB_TABLE']
        
    def database_connection(self):
        try:
            print("Connecting to database...")
            basis = "postgresql://{0}:{1}@{2}:{3}/{4}".format(self.dbusername,
                                                          self.dbpassword, 
                                                          self.dbhost, 
                                                          self.dbport, 
                                                          self.dbname)
            rev_engine = create_engine(basis)
            print("Writing dataframe to table....")
            self.loading_data.to_sql(self.dbtable, con=rev_engine, index=False, if_exists='replace')
            print("Completed!")
            
        except Exception as e:
            print(e)

In [ ]:
loading = Data_Loader(transformed_data)
loading.database_connection()